In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
from sklearn.model_selection import train_test_split

!pip install geopy
from geopy.distance import geodesic
import geopy.distance

In [4]:
file = 'very_clean_delivery.csv'
delivery = pd.read_csv(file, index_col=False)
print(delivery)

       Unnamed: 0  Order_Date Time_Orderd Time_Order_picked Weatherconditions  \
0               0  19-03-2022    11:30:00          11:45:00             Sunny   
1               1  25-03-2022    19:45:00          19:50:00            Stormy   
2               2  19-03-2022    08:30:00          08:45:00        Sandstorms   
3               3  05-04-2022    18:00:00          18:10:00             Sunny   
4               4  26-03-2022    13:30:00          13:45:00            Cloudy   
...           ...         ...         ...               ...               ...   
45588       45588  24-03-2022    11:35:00          11:45:00             Windy   
45589       45589  16-02-2022    19:55:00          20:10:00             Windy   
45590       45590  11-03-2022    23:50:00          00:05:00            Cloudy   
45591       45591  07-03-2022    13:35:00          13:40:00            Cloudy   
45592       45592  02-03-2022    17:10:00          17:15:00               Fog   

      Road_traffic_density 

In [5]:
features = ['Order_Date','Time_Orderd','Time_Order_picked','Weatherconditions','Road_traffic_density','multiple_deliveries','Restaurant_latitude','Restaurant_longitude','Delivery_location_longitude','Delivery_location_latitude']
X = delivery.loc[:, features]
y = delivery.loc[:, ['Time_taken(min)']]

In [6]:
train, test = train_test_split(delivery, test_size=0.25)

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, train_size=0.75)

In [8]:
test = test.replace('NaN', float(np.nan), regex=True)
train = train.replace('NaN', float(np.nan), regex=True)

X_train.head()
X_test.head()

,Order_Date,Time_Orderd,Time_Order_picked,Weatherconditions,Road_traffic_density,multiple_deliveries,Restaurant_latitude,Restaurant_longitude,Delivery_location_longitude,Delivery_location_latitude
25107,19-03-2022,21:25:00,21:40:00,Fog,Jam,1.0,26.913987,75.752891,75.802891,26.963987
20951,13-03-2022,10:55:00,11:05:00,Cloudy,Low,1.0,18.534080,73.898520,73.908520,18.544080
6344,16-02-2022,23:10:00,23:20:00,Sunny,Low,0.0,25.452350,81.841889,81.921889,25.532350
32663,10-03-2022,20:55:00,21:05:00,Windy,Jam,0.0,22.727021,75.884167,75.964167,22.807021
26577,05-03-2022,18:10:00,18:25:00,Fog,Medium,0.0,19.223840,72.841347,72.871347,19.253840


In [9]:
X_train.describe()
y_train.describe()

,Time_taken(min)
count,34194.000000
mean,26.275370
std,9.410603
min,10.000000
25%,19.000000
50%,25.000000
75%,32.000000
max,54.000000


In [10]:
num_cols = ['Restaurant_latitude','Restaurant_longitude','Delivery_location_latitude',
            'Delivery_location_longitude','multiple_deliveries']
for col in num_cols:
    train[col] = train[col].astype('float64')
    
for col in num_cols[:-1]:
    test[col]=test[col].astype('float64')
    
train['Order_Date'] = pd.to_datetime(train['Order_Date'], format="%d-%m-%Y")
test['Order_Date'] = pd.to_datetime(test['Order_Date'], format="%d-%m-%Y")

In [12]:
delivery['Distance_km'] = delivery.apply(lambda x: geopy.distance.distance((x['Restaurant_latitude'], x['Restaurant_longitude']), (x['Delivery_location_latitude'], x['Delivery_location_longitude'])), axis=1)

In [14]:
# Timedelta

In [15]:
train['Time_Orderd'] = pd.to_timedelta(train['Time_Orderd'])
train['Time_Order_picked'] = pd.to_timedelta(train['Time_Order_picked'])

In [16]:
train['Time_Order_picked_updated'] = np.where(train['Time_Order_picked'] < train['Time_Orderd'], 
                                              train['Order_Date'] + train['Time_Order_picked'],
                                             train['Order_Date'] + train['Time_Order_picked'])

train['Time_Orderd_updated'] = train['Order_Date'] + train['Time_Orderd']
train['Order_prep_time_diff'] = ((train['Time_Order_picked_updated'] - 
                                  train['Time_Orderd_updated']).dt.total_seconds())/60

In [17]:
test['Time_Orderd'] = pd.to_timedelta(test['Time_Orderd'])
test['Time_Order_picked'] = pd.to_timedelta(test['Time_Order_picked'])

In [18]:
test['Time_Order_picked_updated'] = np.where(test['Time_Order_picked'] < test['Time_Orderd'], 
                                              test['Order_Date'] + test['Time_Order_picked'],
                                             test['Order_Date'] + test['Time_Order_picked'])

test['Time_Orderd_updated'] = test['Order_Date'] + test['Time_Orderd']
test['Order_prep_time_diff'] = ((test['Time_Order_picked_updated'] - 
                                  test['Time_Orderd_updated']).dt.total_seconds())/60

In [19]:
coords = ['Restaurant_latitude', 'Restaurant_longitude', 'Delivery_location_latitude', 'Delivery_location_longitude']

for coord in coords:
    train[coord] = abs(train[coord])
for coord in coords:
    test[coord] = abs(test[coord])

In [20]:
#train['Distance_km'] = np.zeros(len(train))

#rest_coords_train = train[['Restaurant_latitude', 'Restaurant_longitude']].to_numpy()
#deliver_coords_train = train[['Delivery_location_latitude', 'Delivery_location_longitude']].to_numpy()

#for i in range(len(train)):
   # train['distance_btwn_km'].loc[i] = geodesic(rest_coords_train[i], deliver_coords_train[i])

In [21]:
# train['distance_btwn_km'] = geopy.distance.geodesic(rest_coords_train[i], deliver_coords_train[i])

In [28]:
train = pd.get_dummies(data=train)
test = pd.get_dummies(data=test)

In [29]:
X = train.drop('Time_taken(min)', axis=1)
y = train['Time_taken(min)']

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, random_state=123)